# Trabalho #2 - Detecção de objetos com API do TensorFlow

Nesse trabalho você vai retreinar uma RNA pré-treinada, disponível na API de detecção de objetos do TensorFlow, para detectar novos tipos de objetos em imagens.

### Referências

O código usado nesse trabalho é baseado no Notebook do TensorFlow disponível em https://colab.research.google.com/github/tensorflow/models/blob/master/research/object_detection/colab_tutorials/eager_few_shot_od_training_tf2_colab.ipynb.

## Coloque o seu nome aqui:

Nome:

## 1. Objetivos

O objetivo desse trabalho é realizar um treinamento de detecção de objetos customizado usando uma RNA pré-treinada.

O conjunto de dados consiste de imagens que mostram dois tipos de objetos: 1) cangurus e 2) pessoas. Assim, tem-se um problema de detecção de objetos com duas classes.

A tarefa principal desse trabalho é preparar os dados para treinamento, ou seja, realizar as anotações das imagens. Para realizar essas anotações você vai usar a função `colab_utils.annotate` que usamos na aula. 

Para desenvolver esse trabalho, você vai ter que realizar algumas modificações no progrma usado para essa finalidade visto em aula.

## 2. Instalação de bilbiotecas e da API de detecção de objetos 

### 2.1 Clonar o repostório de modelos do TensorFlow Hub.

In [ ]:
# Importa bibliotecas do sistema operacional
import os
import pathlib

# Clonagem do repositório de modelos do TensorFlow se isso ainda não foi realizado
if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

### 2.2 Importar bibliotecas

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

%matplotlib
%matplotlib inline

### 2.3 Instalar a API de detecção de objetos

In [ ]:
# Instalar API de detecção de objetos se isso ainda não foi realizado
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

### 2.4 Importar funções da API de detecção de objetos.

Após a instalação da API, tem-se disponível bilbiotecas de detecção de objetos com classes e funções que devem ser importadas para o programa para poderem ser usadas.


In [ ]:
# Importa bilbiotecas da API de detecção de objetos
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import config_util 
from object_detection.utils import colab_utils 
from object_detection.builders import model_builder 

- `viz_utils` contém métodos para desenhar caixas delimitadoras.

- `label_map_util` é uma função que associa números para os nomes das classes de objetos. Observa-se que os modelos geram números para as classes e com essa função é possível associar esses números com os nomes das classes.

- `config_util` função para criar modelos a partir do arquivo de configuração

- `colab_utils` função para anotar imagens

- `model_builder` função para construir modelo e carregar parâmetros

### 2.5 Funções auxiliares carregar imagens e desenhar caixas delimitadoras

Na célula abaixo são definidas duas funções, uma para carregar imagens de uma pasta e transformar em tensor Numpy e outra para desenhar as caixas delimitadoras identificadas sobrepostas na imagem. Para visualizar a imagem com as caixas é usado o método `viz_utils.visualize_boxes_and_labels_on_image_array()` da API de detecção de objetos. 

1. `load_img_into_numpy_array()` carrega imagem de uma pasta e a transfroma em tensor Numpy

2. `plot_detection()` mostra uma imagem com as caixas delimitadoras sobrepostas

In [ ]:
# Define função para carregar imagem e transformar em tensor Numpy
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
        path: a file path.

    Returns:
        uint8 numpy array with shape (img_height, img_width, 3)
    """
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

# Define função para mostrar imagem com caixas delimitadoras sobrepostas
def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
    """Wrapper function to visualize detections.

    Args:
        image_np: uint8 numpy array with shape (img_height, img_width, 3)
        boxes: a numpy array of shape [N, 4]
        classes: a numpy array of shape [N]. Note that class indices are 1-based,
                 and match the keys in the label map.
        scores: a numpy array of shape [N] or None.  If scores=None, then
                this function assumes that the boxes to be plotted are groundtruth
        boxes and plot all boxes as black with no classes or scores.
        category_index: a dict containing category dictionaries (each holding
                        category index `id` and category name `name`) keyed by category indices.
        figsize: size for the figure.
        image_name: a name for the image file.
    """
    image_np_with_annotations = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_annotations,
          boxes,
          classes,
          scores,
          category_index,
          use_normalized_coordinates=True,
          min_score_thresh=0.8)
    if image_name:
        plt.imsave(image_name, image_np_with_annotations)
    else:
        plt.imshow(image_np_with_annotations)


## 3. Conjunto de dados

Vamos usar o conjunto de dados originalmente utilizado para detectar cangurus. Porém como muitas imagens desse conjunto mostram também pessoas, vamos utilizá-lo para detectar cangurus e pessoas.

As imagens desse conjunto de dados etá no arquivo compactado `cangurus_pessoas.zip`. Para carregar esse conjutno de dados primeiramente você que que importá-lo para o ambiente do Colab. 

Após importar o arquivo deve descompactá-lo executando a célula abaixo.

In [ ]:
!unzip cangurus_pessoas.zip

A descompactação desse arquivo gera uma pasta de nome "cangurus_pessoas" com duas sub pastas. Uma de nome "train", com as imagens usadas para treinamento, e outra de nome "test" com as imagens usadas para teste.

Verifique no seu ambiente do Colab onde está a pasta "cangurus_pessoas". Para isso execute a célula abaixo.

In [ ]:
! ls

### 3.1 Carregar imagens de treinamento

Para carregar as imagens de treinamento devemos gerar uma lista com os arquivos presentes na pasta "train". 


### Exercício #1: Gerar lista de arquivos das imagens

Para carregar as imagens de treinamemto crie um código na célula abaixo para gerar um alista dos arquivos das imagens.

In [ ]:
# Para você fazer: carregar imagens de treinamento

# Importa bilbliotecas glob
from glob import glob

# Define diretório onde se encontram as imagens
train_dir =

# Escolhe tipos de arquivos desejados
glob_imgs = 

# Cria lista dos nomes dos arquivos
img_paths =

# Mostra 5 primeiros arquivos da lista
print(img_paths[:5])

### Exercício #2: Carregar imagens de treinamento

As imagens de treinamento devem carregas, transformadas em tensores Numpy e colocadas na lista `train_images_np`. Para carregar e transfromar as imagens em tensores Numpy use a função `load_image_into_numpy_array` defina na Seção 2.5.

In [ ]:
# Para você fazer: Carregar e visualizar imagens de treinamento

# Iniciliza lista de imagens
train_images_np = []

# Carrega imagens, tranforma em tensores Numoy e inclui n alista
# Insira seu código aqui
#

# Visualização das imagens
plt.rcParams['axes.grid'] = False
plt.rcParams['xtick.labelsize'] = False
plt.rcParams['ytick.labelsize'] = False
plt.rcParams['xtick.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False
plt.rcParams['ytick.right'] = False
plt.rcParams['figure.figsize'] = [16, 60]

for idx, train_image_np in enumerate(train_images_np):
    plt.subplot(7, 3, idx+1)
    plt.imshow(train_image_np)
plt.show()

- Observa-se que existem 19 imagens no conjunto de treinamento. Esse número é muito pequeno para realizar um treinamento efetivo, porém a anotação dessas 19 imagens é bastante trabalhosa.

### 3.2  Anotação das imagens com as caixas delimitadoras

Você tem que anotar as imagens identificando os cangurus e as pessoas. Com visto, isso representa desenhar uma caixa ao redor de cada canguru e cada pessoa presentes nas imagens. Ao fazer isso, são criadas as caixas delimitadoras desejadas para cada imagem, que representam as saídas desejadas.

A API de detecção de objetos disponibiliza a função `colab_utilis.annotate()` para auxiliar a execução dessa tarefa. Infelizmente essa função é bastante limitada e não permite definir as classes de cada caixa, portanto, isso terá que ser realizado após serem definidas todas as caixas de todas as imagens.

Oberva que existem muitos softwares para facilitar a anotação de imagens, um bstante completo e de uso livre para poucas iamgens é o Roboflow (https://roboflow.com/).

**Importante:** a qualidade do seu modelo de detecção de objetos depende muito da realização de um trabalho de anotação das imagens. Dessa forma, tente ser o mais preciso possível nessa tarefa.


### Exercício #3; Anotação das imagens de treinamento

Execute a célula abaixo para realizar as anotações das 19 imagens de treinamento.

#### Dicas:

1. Utlize classe 0 para os cangurus e classe 1 para as pessoas.
2. Para facilitar a definição das classes de cada caixa de cada imagem, para cada imagem defina primeiro todas as caixas dos cangurus para depois definir as caixas das pessoas. Isso vai facilitar associar para cada caixa de cada imagem a sua classe correspondente.
3. Após criar a lista de classes é interessante copiá-la para poder usá-l posteriormenet e assim, evitar de ter que repetir essa tarefa de anotação das imagens toda vez que tiver que refazer o trabalho.

In [ ]:
# Para você fazer: Anotação das imagens

# Inicializar a lista de caixas
gt_boxes = []

# Anotação manual das imagens para criar caixas
# Insira seu código aqui
#


A lista de caixas identificadas nas imagens é criada na variável `gt_boxes` e é apresentada executando a célula abaixo.

In [ ]:
# Apresenta lista de caixas geradas na anotação das imagens
gt_boxes

### Exercíco #4: Definição das classes das caixas

Você tem que associar cada caixa presente nas 19 imagens com a sua classe e criar uma lista com as classes de todas as imagens.

As classes são representadas por números inteiros. No caso de se ter duas classes, pode-se definir por exemplo:

- classe = 0, para canguru;
- classe = 1, para pessoa.

Para cada imagem deve ser associada um vetor com as classes de cada caixa. Por exemplo, se a imagem 1 tem três caixas, sendo duas de cangurus e uma de pessoa, para essa imagem o vetor de classes deve ser o seguinte: `[0, 0, 1]`.

A lista de classes é obtida simplesmente unindo os vetores de classes de cada imagem em uma variável tipo lista usando o método `append`. 


In [ ]:
# Para você fazer: Definição das classes das caixas

# Incicializar lista de classes das imagens
classes=[]

# Definir vetor de classe de cada imagem e incluir na lista de classes
# Insira seu código aqui
#

# Mostra lista de classes
print(classes)

### 3.3 Preparar dados para o treinamento

Os dados de treinamento devem ser preparados. Essa preparação envolve as seguintes etapas:

1. Criar dicionário que associa número das classes com os seus nomes.
2. Converter a imagens para tensor do TensorFlow e em números reais, e incluir eixo dos exemplos. Observa-se que a normalização das imagens é realizada dentro da rede.
3. Converter lista de caixas para tensor do TensorFlow.
4. Converter lista de classes para tensor do TensorFlow.
5. Converter classes para vetores one-hot.


### Exercício #5: Preparar dados

Na célula abaixo execute as etapas de preparação dos dados descritas acima.

Dicas:
- Para converter uma imagem ou um vetor em um tensor TF use a função `tf.convert_to_tensor()`.
- Ao converter uma imagem ou um vetor em tensor TF deve definir o tipo de dado, para as imagens e caixas usar `tf.float32` e para as classes `tf.int32`. 
- Para incluir o eixo dos exemplos nas imagens use a função `tf.expand_dims()`.

Observa-se que não precisa usar a variável `label_id_offset`, como usado na aula, se identificou as classes com 0 e 1.

In [ ]:
# Para você fazer: Preparação dos dados de treinamento

# Define número de classes
num_classes = 2

# Define dicionário com números e nomes das classes
# Insira seu código aqui
#


# Os índices das classes devem ser convertidos para vetores one-hot e todas as variáveis devem ser
# transformadas em tensores.

# Incializa lista de imagens com números reais
train_image_tensors = []

# Inicializa lista de classes codificadas com vetores one-hot
gt_classes_one_hot_tensors = []

# Inicializa lista de caixas 
gt_box_tensors = []

# Iterage em todas as imagens de treinamento incluindo as imagens, as caixas e as classes transfromadas nas listas
for (train_image_np, gt_box_np, classe) in zip(train_images_np, gt_boxes, classes):
    
    # Incluiu eixo dos exemplos e transforma imagens em tensores com números reais (use axis=0)
    train_image_tensors.append
    
    # Transforma caixas a serem previstas em tensores TF
    gt_box_tensors.append
    
    # Transforma caixas reais em tensores TF
    zero_indexed_groundtruth_classes = 

    # Codificação one-hot das classes
    gt_classes_one_hot_tensors.append

print('Preparação de dados pronta')

Visualização das classe codificadas em vetores one-hot.

In [ ]:
print(gt_classes_one_hot_tensors)
x = np.array(classes[0])
print(x, x.shape)

### 3.4 Visualização das imagens de treinamento com as caixas sobrepostas

Para verificar se a preparação de dados foi realizada de forma correta, é importante visualizar as imagens com as caixas sobrepostas indentificadas pelas suas classes e pela probabilidade de estarem corretas.

Para realizar essa visualização é necessário primeiramente definir uma lista de probabilidades ("scores") "fake", porque a função de visualização disponibilizada pelo TensorFlow exige esse dado de entrada.


### Exercíco #6: Definição dos "scores"

Para poder visualizar as imagens de treinamento com as caixas sobrepostas deve-se associar cada caixa presente nas 19 imagens de treinamento com a probabilidade da caixa estar identificando corretamente um objeto. Além disso, deve-se criar uma lista com as probabilidades de todas as caixas das imagens.

O formato dessa lista de probabilidades é o mesmo usado para definir as classes de cada caixa. Contudo, as probabilidades são números reais variando de 0.0 a 1.0. Como essas probabilidades são somente para permitir visualizar as imagens de treinamento com as caixas deve-se definir todas as probabilidades iguais a 1.0. 

In [ ]:
# Para você fazer: Defir lista de probabilidades das caixas

# Inicializa lista de probabilidades
dummy_scores=[]

# Define vetor de probabilidade das caixas de cada imagem e une a lista, segundo o exemplos
# score1 = [1., 1.]
# dummy_scores.append(score1)
# Insira seu código aqui
#

# Apresenta resultados
print(dummy_scores)

Execute a célula abaixo para visualizar as imagens de treinamento com as caixas sobrepostas.

In [ ]:
plt.figure(figsize=(16, 60))
for idx, train_image_np in enumerate(train_images_np):
    plt.subplot(2, 3, idx+1)
    plot_detections(train_images_np[idx], gt_boxes[idx], classes[idx], dummy_scores[idx], category_index)
plt.show()

## 4. Criar modelo pré-treinado e restaurar seus parâmetros 

Nesta seção deve-se construir o modelo de detecção de objetos do tipo RetinaNet e carregar os seus parâmetros pré-treinados. 

Nesse trabalho recomenda-se usar o mesmo modelo usado da aula de detecção de objetos com a API do tensorFlow, ou seja, o modelo SSD-ResNet50 para imagens com dimensão 640x640x3. Esse modelo utiliza o método SSD para detecção dos objejos, usando a rede ResNet-50 para extrair as características das imagens. Se quiser pode tentar usar outro modelo, mas cuidado porque você pode ter dificuldade para configurá-lo e criá-lo.

Execute a célula abaixo para carregar os parâmetros do modelo e os colocá-los na pasta do Colab "content/research/models/detection/test_data".

Observa-se que se você escolheu usar outro modelo é necessário alterar o código das duas células que realizam essas operações de criar o modelo e carregar seus parâmetros.

In [ ]:
# Carrega o arquivo de parâmetros ("checkpoint") e o coloca na pasta 
# models/research/object_detection/test_data/SSD_ResNet_50
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint models/research/object_detection/test_data/

Para criar o modelo é usado o arquivo de configuração, que define a arquitetura do modelo, e os parâmetros pré-treinados ("checkpoints").

Ao executarmos a célula acima carregamos o arquivo de configuração e os parâmetros pré-treinados do modelo que iremos usar. 

- Arquivo de configuração: "models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config"
- Prefixo dos arquivos com parâmetros pré-treinados do modelo: "models/research/object_detection/test_data/checkpoint/ckpt-0"

Se abrir o arquivo de configuração é possível ver a configuração do modelo.

<br>

Execute a  célula abaixo para criar o modelo.

In [ ]:
tf.keras.backend.clear_session()
print('Construindo modelo e carregando parâmetros pré-treinados...', flush=True)

# Define número de classes do novo conjunto de dados
num_classes = 2

# Define arquivos de configuraçao e prefixo dos arquivos de parâmetros
pipeline_config = 'models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
checkpoint_path = 'models/research/object_detection/test_data/checkpoint/ckpt-0'

# Define diretório para slavar arquivo de configuração e parãmetros do modelo retreinado
output_directory = 'output/'
output_checkpoint_dir = os.path.join(output_directory, 'checkpoint')

# Alteração do arquivo de configuração 
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
model_config.ssd.num_classes = num_classes
model_config.ssd.freeze_batchnorm = True

# Construção do modelo completo com parâmetros inicializados aleatóriamente
detection_model = model_builder.build(model_config=model_config, is_training=True)

# Salva novo arquivo de configuração 
pipeline_proto = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_proto, output_directory)

# Cria subrede de identificação de caixas usando a subrede do modelo pré-treinado
fake_box_predictor = tf.compat.v2.train.Checkpoint(
    _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
    # Se for desejado deixar também a subrede de classificação, deve-se tirar o comentário da linha abaixo
    # _prediction_heads=detection_model._box_predictor._prediction_heads,
    _box_prediction_head=detection_model._box_predictor._box_prediction_head,
    )

# Cria modelo parcial com rede de extração de caracteríticas e subrede de identificação das caixas 
fake_model = tf.compat.v2.train.Checkpoint(
             _feature_extractor=detection_model._feature_extractor,
             _box_predictor=fake_box_predictor)

# Carrega parâmetros pré-treinados no modelo parcial, ou seja, na rede de extração 
# de característica e na subrede de identificação das caixas
ckpt = tf.compat.v2.train.Checkpoint(model=fake_model)
ckpt.restore(checkpoint_path).expect_partial()

# Transfere modelo parcial com parâmetros pré-treinados para modelo final
exported_ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt_manager = tf.train.CheckpointManager(exported_ckpt, output_checkpoint_dir, max_to_keep=1)

# Executa modelo com imagem dummy para inicializar variáveis criadas
image, shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))
prediction_dict = detection_model.predict(image, shapes)
_ = detection_model.postprocess(prediction_dict, shapes)

print('Modelo criado!')

Mostra novo arquivo de configuração para inspeção.

In [ ]:
model_config

## 5. Treinamento do modelo

A célula abaixo define os parâmetros do modelo que será retreinado e executa o treinamento. 


### Exercício #7: Seleção dos parâmetros de treinamento

Na ceúla abaixo você deve definir os seguintes parâmetros de treinamento:

- `batch_size` - define tamanho do lote;
- `learning_rate` - define taxa de aprendizado;
- `num_batches` - define quantos lotes são utilizados no treinamento;
- Otimizador a ser usado.

Observe que o lopp de treinamento se repete em função do número de lotes. Por exemplo, se existirem 100 exemplos de treinamento e os lotes (`batch_size`) forem de 25 exemplos, tem-se 4 lotes de exemplos. Assim, se o número de lotes (`num_batches`) for igual a 200, então cada lote de dados será utlizado no treinamemto 50 vezes.

Após definir esses parâmetros basta executar a célula para realizar o treinamento.

In [ ]:
# Para você fazer: Seleção de parâmetros de treinamemto

tf.keras.backend.set_learning_phase(True)

# Define tamanho do lote de treinamento
batch_size = 

# Define taxa de treinamento
learning_rate =

# Define número de lotes usados no treinamento
num_batches = 


# Define parâmetros do modelo que são alterados no treinamento
trainable_variables = detection_model.trainable_variables
to_fine_tune = []
prefixes_to_train = [
    'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead',
    'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead']
for var in trainable_variables:
    if any([var.name.startswith(prefix) for prefix in prefixes_to_train]):
        to_fine_tune.append(var)

        
# Define função para carregar modelo e otimizador para treinamento
def get_model_train_step_function(model, optimizer, vars_to_fine_tune):
    """Get a tf.function for training step."""

    # Define função para executar um passo de treinamento que processa um lote de dados
    # Função é criada na forma tf.function para aumentar velocidade de computação
    # Pode-se comentar a linha @tf.function para desabilitar essa opção
    #@tf.function
    def train_step_fn(image_tensors,
                      groundtruth_boxes_list,
                      groundtruth_classes_list):
        """A single training iteration.

        Args:
        image_tensors: A list of [1, height, width, 3] Tensor of type tf.float32.
             Note that the height and width can vary across images, as they are
             reshaped within this function to be 640x640.
        groundtruth_boxes_list: A list of Tensors of shape [N_i, 4] with type
             tf.float32 representing groundtruth boxes for each image in the batch.
        groundtruth_classes_list: A list of Tensors of shape [N_i, num_classes]
             with type tf.float32 representing groundtruth boxes for each image in
             the batch.

        Returns:
            A scalar tensor representing the total loss for the input batch.
        """
        # Define dimensão do tensor com dados e entrada
        shapes = tf.constant(batch_size * [[640, 640, 3]], dtype=tf.int32)
        
        # Obtém saídas reais 
        model.provide_groundtruth(
              groundtruth_boxes_list=groundtruth_boxes_list,
              groundtruth_classes_list=groundtruth_classes_list)
        
        # Calcula gradiente da função de custo em relação aos parâmetros
        with tf.GradientTape() as tape:
            # Pré-processa imagens do lote
            preprocessed_images = tf.concat([detection_model.preprocess(image_tensor)[0]
                                             for image_tensor in image_tensors], axis=0)
            
            # Executa modelo para obter previsões das classes e caixas
            prediction_dict = model.predict(preprocessed_images, shapes)
            
            # Calcula função de custo
            losses_dict = model.loss(prediction_dict, shapes)
            total_loss = losses_dict['Loss/localization_loss'] + losses_dict['Loss/classification_loss']
            
            # Calcula gradiente da função de custo em relação aos parâmetros treináveis
            gradients = tape.gradient(total_loss, vars_to_fine_tune)
            
            # Aplica otimizador para atualizar parãmetros
            optimizer.apply_gradients(zip(gradients, vars_to_fine_tune))
            
        return total_loss
    return train_step_fn

# Define otimizador (SGD com momento) 
# Insira seu código aqui
# optimizer = 

# Inicializa função de treinamento passando modelo, otimizador e lista de parâmetros treináveis
train_step_fn = get_model_train_step_function(detection_model, optimizer, to_fine_tune)

print('Inicio do treinamento!', flush=True)

# Executa loop de treinamento 
for idx in range(num_batches):
    # Embaralha dados aleatóriamente
    all_keys = list(range(len(train_images_np)))
    random.shuffle(all_keys)
    example_keys = all_keys[:batch_size]

    # Obtém dados de treinamenteo 
    gt_boxes_list = [gt_box_tensors[key] for key in example_keys]
    gt_classes_list = [gt_classes_one_hot_tensors[key] for key in example_keys]
    image_tensors = [train_image_tensors[key] for key in example_keys]

    # Realiza um passo de treinamento
    total_loss = train_step_fn(image_tensors, gt_boxes_list, gt_classes_list)

    if idx % 10 == 0:
        print('batch ' + str(idx) + ' of ' + str(num_batches) + ', loss=' +  str(total_loss.numpy()), flush=True)

# Salva novos parâmetros do modelo         
ckpt_manager.save()

print('Término do treinamento!')

## 7. teste do modelo

Para testar o modelo retreinado com o novo conjunto de dados vamos usar as imagens de teste.


### Exercício #8: Gerar lista de arquivos das imagens de teste

Para carregar as imagens de treinamento deve-se gerar uma lista com os arquivos presentes na pasta "cangurus_pessoas/test". Crie um código na célula abaixo para gerar uma lista dos arquivos das imagens de teste.

In [ ]:
# Para você fazer: Criar lista de arquivos das imagens de teste

# Define diretório onde se encontram as imagens
test_dir = 

# Escolhe tipos de arquivos desejados
glob_imgs = 

# Cria lista dos nomes dos arquivos
img_paths = 

# Número de imagens de teste
print(len(img_paths))

# Inicializa lista de imagens de teste
test_images_np = []

# Le arquivos da imagens e inclui dados na lista de imagens
# Insira seu código aqui
#


# Mostra imagens de teste
plt.rcParams['axes.grid'] = False
plt.rcParams['xtick.labelsize'] = False
plt.rcParams['ytick.labelsize'] = False
plt.rcParams['xtick.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False
plt.rcParams['ytick.right'] = False
plt.rcParams['figure.figsize'] = [16, 60]

for idx, test_image_np in enumerate(test_images_np):
    plt.subplot(6, 5, idx+1)
    plt.imshow(test_image_np)
plt.show()

Execute a célula abaixo para criar a a função `detect()`, que recebe uma imagem na forma de tensor TF e calcula as previsões de objetos presentes na imagem.

In [ ]:
# Cria função para calcular previsões do modelo
# Função é criada na forma tf.function para aumentar velocidade de computação
# Pode-se comentar a linha @tf.function para desabilitar essa opção
@tf.function
def detect(input_tensor):
    """Run detection on an input image.

    Args:
        input_tensor: A [1, height, width, 3] Tensor of type tf.float32.
        Note that height and width can be anything since the image will be
        immediately resized according to the needs of the model within this
        function.

    Returns:
       A dict containing 3 Tensors (`detection_boxes`, `detection_classes`, and `detection_scores`).
    """
    # Pré-processa imagens
    preprocessed_image, shapes = detection_model.preprocess(input_tensor)
    # calcula previsões do modelo
    prediction_dict = detection_model.predict(preprocessed_image, shapes)
    # Retorna previsões pós-processadas
    return detection_model.postprocess(prediction_dict, shapes)

### Exercício #9: Calculo das pervisões do modelo

Complete a célula abaixo e depois a execute para calcular as previsões do seu modelo e mostrar as imagens com as caixas detectadas sobrepostas.

Note que a saída da função `detect()` é um dicionário que contém as seguintes `keys`:

    detections['detection_boxes'];
    detections['detection_classes'];
    detections['detection_scores'].


In [ ]:
# Para você fazer: Preparação das imagens de cálculo e cálculo das previsões

# Tamanho de apresentação das imagens 
plt.figure(figsize=(15, 60))

# Loop para executar previsões do modelo nas imagens de teste
for i in range(len(test_images_np)):

    # Transforma imagens em tensor do TF de números reais
    input_tensor = 
    
    # Inclui eixo dos exemplos nas imagens
    input_tensor = )
    
    # Calcula previsões
    detections = 

    # Chama função plot_detections() para criar imagens com  as caixas
    plt.subplot(9, 3, i+1)
    # Insira seu código aqui
    #
    
plt.show()

## 8. Utilização do modelo treinado

Como visto em aula, o modelo criado utiliza camadas customizadas e o método de programação em "graphos" do TensorFlow de forma que para ser utilizado após ter sido salvo ele tem que ser reconstruído. O modelo salvo é reconstruído a partir do arquivo de configuração e dos parâmetros salvos. Assim, se quiser reutilizar esse modelo você deve seguir os passos apresentados em aula para fazer isso.